# Imports

In [31]:
import blocksnet as bn
%matplotlib inline
from shapely.geometry import Point
from geopandas import GeoDataFrame
from blocksnet.models.land_use import LandUse

In [32]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Model

## Читаем модель

In [33]:
city = bn.City.from_pickle(r'C:\Users\rkozl\Desktop\BlocksNet\blocksnet\small_model.pickle')

Выбираем квартал который хотим менять и выбираем новое землепользование для него

In [34]:
blocks_landuse_dict ={
    104:LandUse.TRANSPORT,
    10:LandUse.RESIDENTIAL
}

scenario = {
    'substation': 0.9
}

# Optimizer

Инициализируем класс для подбора оптимальной застройки сервисов в соответсвии с выбранным землепользование

In [35]:
opt = bn.BlockOptimizer(city_model=city)
result = opt.calculate(blocks_landuse_dict, scenario, max_facilities=None)

Optimal


Метод выдает оптимальные типы сервисов и их вместимость

In [36]:
result['optimal_update_df']

,substation,bus_stop
104,18980.0,0.0
10,3930.0,4.0


In [37]:
from pprint import pprint
for key in result['bricks_to_build_df']:
    print(f"block index {key}")
    if 'population' in result['optimal_update_df'].columns:
        print(result['optimal_update_df']['population'].loc[key])
    result['bricks_to_build_df'][key]['coef'] = result['bricks_to_build_df'][key]['capacity_agg']/result['bricks_to_build_df'][key]['demand_in_need']
    area = int(result['bricks_to_build_df'][key]['area_agg'].sum())
    area_2 = int(city[key].site_area)
    print(f"filled area {area}/{area_2} coef {area/area_2}")
    print(result['bricks_to_build_df'][key])
    print()

block index 104
filled area 379600/379638 coef 0.9998999046460049
   service_type  capacity   area  capacity_agg  area_agg  service_counts  demand_in_need      coef
21   substation         5  100.0       18980.0  379600.0          3796.0          166486  0.114004

block index 10
filled area 78607/78629 coef 0.9997202050134174
    service_type  capacity   area  capacity_agg  area_agg  service_counts  demand_in_need      coef
171   substation         5  100.0        3930.0   78600.0           786.0          166116  0.023658
177     bus_stop         4    7.0           4.0       7.0             1.0               5  0.800000



Если изменять значение в сценарии можно контролировать количество сервисов

In [38]:
blocks_landuse_dict ={
    104:LandUse.TRANSPORT,
    10:LandUse.RESIDENTIAL
}

scenario = {
    'substation': 0.01
}

In [39]:
opt = bn.BlockOptimizer(city_model=city)
result = opt.calculate(blocks_landuse_dict, scenario, max_facilities=None)

Optimal


In [40]:
opt = bn.BlockOptimizer(city_model=city)
result = opt.calculate(blocks_landuse_dict, scenario, max_facilities=None)
from pprint import pprint
for key in result['bricks_to_build_df']:
    print(f"block index {key}")
    if 'population' in result['optimal_update_df'].columns:
        print(result['optimal_update_df']['population'].loc[key])
    result['bricks_to_build_df'][key]['coef'] = result['bricks_to_build_df'][key]['capacity_agg']/result['bricks_to_build_df'][key]['demand_in_need']
    area = int(result['bricks_to_build_df'][key]['area_agg'].sum())
    area_2 = int(city[key].site_area)
    print(f"filled area {area}/{area_2} coef {area/area_2}")
    print(result['bricks_to_build_df'][key])
    print()

Optimal
block index 104
filled area 379630/379638 coef 0.9999789272938958
     service_type  capacity    area  capacity_agg  area_agg  service_counts  demand_in_need      coef
13       dog_park       106   400.0         106.0     400.0             1.0           112.0  0.946429
21     substation         5   100.0        8450.0  169000.0          1690.0        166486.0  0.050755
25  train_station        40   650.0          40.0     650.0             1.0          6348.0  0.006301
26  train_station       100  1300.0        6300.0   81900.0            63.0          6348.0  0.992439
27    bus_station        70   150.0         630.0    1350.0             9.0         77939.0  0.008083
28    bus_station       100   160.0         500.0     800.0             5.0         77939.0  0.006415
29    bus_station       200   320.0       76800.0  122880.0           384.0         77939.0  0.985386
33           pier        50    50.0        2650.0    2650.0            53.0          2679.0  0.989175

block i

Также можно узнать какие сервисы были снесены перед постройкой новых

In [42]:
result['deleting_df']

,cemetery,embankment,park,fuel,religion,hospital,museum,university,market,population,pitch,restaurant,convenience,bank,lawyer,cafe,multifunctional_center,hotel,parking
0,-207.0,-700.0,-290,-740.0,-1172.0,-6062.0,-6620.0,-7429.0,-734.0,-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,-278,0.0,0.0,0.0,0.0,0.0,0.0,-1175,-28.0,-540.0,-511.0,-63.0,-250.0,-45.0,-2705.0,-655.0,-18.0


И посмотреть число обеспеченности

In [43]:
result['provision']

-48140.896538461544

С помощью метода plot мы можем отобразить гистограмму изменений в в квартале по сравнению с оригинальной застройкой

In [ ]:
opt.plot(**result)

Также можно выбрать оптимальное землепользование использовав метод для всех возможных типов землепользования

In [ ]:
best_landuse = selected_block.land_use
temp_diff = 0
for lanuse in LandUse:
    if lanuse == orig_landuse:
        continue
    result = opt.calculate(selected_block=selected_block, new_landuse=lanuse)
    diff = result['total_after'] - result['total_before']
    print(f'{selected_block.land_use} -> {lanuse}: {round(diff, 3)}')
    if diff > temp_diff:
        temp_diff = diff
        best_landuse = lanuse

print(f'best lanuse {best_landuse}')